# RAG Advanced Retrieval


## Notebook Setup

In [2]:
# Importing the necessary Python libraries
import os
import json
import time
import yaml

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from datasets import Dataset
from langchain.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_community.document_loaders import DataFrameLoader
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_recall,
    context_precision
)

/home/vscode/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/vscode/.local/lib/python3.11/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in _VertexAIBase has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/home/vscode/.local/lib/python3.11/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in _VertexAICommon has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/home/vscode/.local/lib/python3.11/site-packages/ragas/metrics/__init__.py:4: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pyda

In [3]:
# Loading in the data from the CSV files
df_kis = pd.read_csv('../data/synthetic_knowledge_items.csv')
df_validation = pd.read_csv('../data/rag_sample_qas_from_kis.csv')

# Dropping alt_ki_text from the df_kis DataFrame
df_kis.drop(columns = ['alt_ki_text'], inplace = True)

# Dropping any unnecessary columns from the validation DataFrame
df_validation.drop(columns = ['ki_topic', 'ki_text'], inplace = True)

# Renaming the remaining columns
df_validation.rename(columns = {
    'sample_question': 'question',
    'sample_ground_truth': 'ground_truth'
}, inplace = True)

In [9]:
# Setting the embedding algorithm
embedding_algorithm = OpenAIEmbeddings(model = 'text-embedding-3-large')

# Setting the chat model
chat_model = ChatOpenAI(model = 'gpt-4o')

## Vector Database Setup
Since we already covered advanced ingestion techniques in a different notebook, we'll quickly set ourselves up here with a vector database that we can use for practicing our advanced retrieval techniques.

In [5]:
# Setting the filepath for the index file
index_file = '../data/semantic_index.bin'

# Checking if the index file exists
if os.path.exists(index_file):

    # Load the index from file
    faiss_index = FAISS.load_local(index_file,
                                   embeddings = OpenAIEmbeddings(),
                                   allow_dangerous_deserialization = True)

# Creating the FAISS index from scratch
else:

    # Loading the documents
    documents = DataFrameLoader(df_kis, page_content_column = 'ki_text').load()

    # Creating a semantic text splitter
    text_splitter = SemanticChunker(embeddings = embedding_algorithm)

    # Splitting the documents into chunks
    chunks = text_splitter.split_documents(documents)

    # Creating FAISS index for the current chunk size
    faiss_index = FAISS.from_documents(chunks, embedding_algorithm)

    # Save the index to file
    faiss_index.save_local(index_file)

In [6]:
# Getting the number of documents in the FAISS index
num_documents = faiss_index.index.ntotal

# Printing the number of documents
print(f'Number of documents in the FAISS index: {num_documents}')

Number of documents in the FAISS index: 269


## Advanced Retrieval Techniques
Let's move into talking about the advanced retrieval techniques we'll use! For your own use case, you may want to use a different combination of these various techniques. Keep in mind: some of these techniques WILL add latency your pipeline. In addition to applying these techniques to get better results, you will want to ensure that you are balancing the trade-offs between speed and accuracy.

### HyDE (Hypothetical Document Embeddings)
Don't let the name fool you: this is actually a relatively simple technique. The idea is to take a query and generate a hypothetical document embedding for it. This hypothetical document embedding is then used to retrieve documents from the vector database. This is a great technique to use when you have a query that is very different from the documents in your vector database.

In [8]:
dd

In [ ]:
# Setting up a prompt to help us produce the HyDE generation
HyDE_PROMPT = '''Generate a brief, factual paragraph that answers the following question: {query}'''

# Setting up the model to use for HyDE generation
hyde_model = ChatOpenAI(model = 'gpt-4o-mini')